In [1]:
from pyspark.sql import SparkSession

from pyspark.sql.types import StructType, IntegerType , StringType
from pyspark.sql.functions import *

In [2]:
spark=SparkSession\
        .builder\
        .appName('Streamed JSON records to PostgreSQL')\
        .config("spark.jars", "postgresql-42.5.4.jar")\
        .getOrCreate()

Picked up _JAVA_OPTIONS: -Dawt.useSystemAAFontSettings=on -Dswing.aatext=true
Picked up _JAVA_OPTIONS: -Dawt.useSystemAAFontSettings=on -Dswing.aatext=true


23/02/18 10:08:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
schema=StructType()\
    .add("id",IntegerType())\
    .add("name",StringType())\
    .add("host_id", IntegerType())\
    .add("neighbourhood_cleansed",StringType())\
    .add("price",StringType())

In [4]:
lines=spark\
        .readStream\
        .format('socket')\
        .option('host', 'localhost')\
        .option('port', 9999)\
        .load()\
        .select(from_json(col("value").cast("string"),schema).alias("json"))\
        .select(col("json.*"))
        


23/02/18 10:08:43 WARN TextSocketSourceProvider: The socket source should not be used for production applications! It does not support recovery.


In [5]:
transformation=lines.withColumn('price', regexp_replace('price','([$])',''))\
                    .withColumn("price" , col("price").cast(IntegerType()))\
                    .withColumnRenamed("neighbourhood_cleansed","neighbourhood")

In [6]:
transformation.printSchema()

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- host_id: integer (nullable = true)
 |-- neighbourhood: string (nullable = true)
 |-- price: integer (nullable = true)



In [7]:
from pyspark.sql import *
def load_psql(row, epochId):
    row.write\
        .format("jdbc") \
        .option("url", "jdbc:postgresql:database1") \
        .option("dbtable", "spark") \
        .option("user", "kali")\
        .option("password", "kali")\
        .option("driver","org.postgresql.Driver")\
        .mode("append")\
        .save()
        
       
       

In [9]:
query=transformation\
        .writeStream\
        .foreachBatch(load_psql)\
        .start()


23/02/18 10:18:53 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-050bfe70-b7d9-499f-91ab-2900783a400c. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
23/02/18 10:18:53 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [10]:
query.stop()

In [11]:
spark.stop()